# Prototyping and Testing the execution of the verifiers on the dataset

#### Some config

In [ ]:
NUM_PROC = 1

## Execution Code

In [ ]:
from vif.baselines.models import VerEvaluation
from vif.baselines.verifiers_baseline.ver_baseline import TexVerBaseline
from datasets.formatting.formatting import LazyBatch
from PIL import Image
import sys
from datasets import load_dataset,Dataset
from loguru import logger

from vif.utils.renderer.tex_renderer import TexRenderer

renderer = TexRenderer()


def execute_verifier_on_dataset(
    verifier: TexVerBaseline, dataset: Dataset
):
    def exec_verif(row):
        metadata = verifier.get_config_metadata()
        ver_eval_input: VerEvaluation = VerEvaluation(
            approach_name=metadata["name"],
            config_metadata=metadata,
            initial_code=row["code"],
            initial_image=row["original_image"],
            initial_instruction=row["instruction"],
            initial_solution=row["solution"],
            initial_solution_image=row["solution_image"],
            expected=row["expected"]
        )
        result = verifier.assess_customization(ver_eval_input)
        result_row = result.model_dump()
        result_row["id"] = row["id"]
        return result_row

    return dataset.map(exec_verif, num_proc=NUM_PROC)

In [ ]:

logger.configure(handlers=[{"sink": sys.stdout, "level": "WARNING"}])
ds = load_dataset("CharlyR/VeriTikz", "full", split="train")

## Execution of the verifiers

#### Config

In [ ]:
import openai
import os
from google import genai
from google.genai import types as genTypes
from vif.baselines.verifiers_baseline import (
    FalconVerifier,
    TextVerifier,
    TextVisualVerifier,
    ViperGPTVerifier,
    VisualPropertiesVerifier,
    VisualVerifier,
    VisualCodeVerifier,
)


client = openai.Client(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)

gclient = genai.Client(
        api_key=os.environ.get("GOOGLE_API_KEY"),
        http_options=genTypes.HttpOptions(api_version="v1alpha"),
    )


#### Text Verifier

In [ ]:
text_verifier: TexVerBaseline = TextVerifier(
    model="qwen/qwen2.5-vl-32b-instruct:free",
    temperature=0.5,
    client=client
)

verifier_text_ds = execute_verifier_on_dataset(text_verifier,ds)

#### Visual Verifier

In [ ]:
visual_verifier: TexVerBaseline = VisualVerifier(
    model="qwen/qwen2.5-vl-32b-instruct:free",
    temperature=0.5,
    client=client
)

verifier_visual_ds = execute_verifier_on_dataset(visual_verifier,ds)

#### Text/Visual Verifier

In [ ]:
text_visual_verifier: TexVerBaseline = TextVisualVerifier(
    model="qwen/qwen2.5-vl-32b-instruct:free",
    temperature=0.5,
    client=client
)

verifier_text_visual_ds = ds.map(execute_verifier_on_dataset(text_visual_verifier,ds))

#### Visual Verifier with code

In [ ]:
text_visual_code_verifier: TexVerBaseline = VisualCodeVerifier(
    model="qwen/qwen2.5-vl-32b-instruct:free",
    temperature=0.5,
    client=client
)

verifier_visual_code_ds = ds.map(execute_verifier_on_dataset(text_visual_code_verifier,ds))

#### Visual Property Verifier

In [ ]:
visual_property_verifier: TexVerBaseline = VisualPropertiesVerifier(
    model="qwen/qwen2.5-vl-32b-instruct:free",
    temperature=0.5,
    client=client
)

verifier_visual_property_ds = ds.map(execute_verifier_on_dataset(visual_property_verifier,ds))

#### ViperGPT Verifier

In [ ]:
#full config available here in config file vif/baselines/verifiers_baseline/ViperGPT_adapt/ViperGPT_config.py
visual_property_verifier: TexVerBaseline = ViperGPTVerifier(
    model="qwen/qwen2.5-vl-32b-instruct:free",
    temperature=0.5,
    client=client
)

verifier_vipergpt_ds = ds.map(execute_verifier_on_dataset(visual_property_verifier,ds))

#### Ours

In [ ]:
falcon_verifier: TexVerBaseline = FalconVerifier(
    oracle_gen_model="qwen/qwen2.5-vl-32b-instruct:free",
    vision_model="gemini-2.5-flash",
    property_model="qwen/qwen2.5-vl-32b-instruct:free",
    property_model_temperature=0.5,
    gclient=gclient,
    oclient=client
)

verifier_falcon_ds = ds.map(execute_verifier_on_dataset(falcon_verifier,ds))